In [1]:
'''
imports
'''

#standard utilities
import pandas as pd
import numpy as np
import os
import random
from collections import Counter
import itertools
import time

#plots
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn import svm 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, auc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer




from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from gensim.models import FastText
import fasttext

In [2]:
#df = pd.read_parquet("data/TweetsCOV19_preprocessed.parquet")
glove_file = 'data/glove.twitter.27B.25d.txt'  # Path to the GloVe file
wv_glove = KeyedVectors.load_word2vec_format(glove_file, binary=False)
default_glove = np.zeros(25, dtype=np.float32)

In [3]:
word2vec_file = 'data/GoogleNews-vectors-negative300.bin'
wv_word2vec = KeyedVectors.load_word2vec_format(word2vec_file, binary=True, limit=2000000)
default_word2vec = np.zeros(300, dtype=np.int32)

In [4]:
FastText_file = 'data/cc.en.300.bin'
wv_fasttext = fasttext.load_model(FastText_file)
wv_fasttext["hello"].shape

(300,)

In [5]:
'''
Vader Q1
'''

'\nVader Q1\n'

In [6]:
'''
Vader Q2
'''

'\nVader Q2\n'

In [7]:
dftr = pd.read_parquet("data/train.parquet")
dfte = pd.read_parquet("data/test.parquet")
dfvl = pd.read_parquet("data/valid.parquet")
df = pd.concat([dftr, dfte])

max_words = sorted([len(string.split(' ')) for string in df['X'].values], reverse=True)
print(max_words[0:100])
max_words = max_words[0]
#[word for string in df['X'].values for word in string.split(' ')]

unique_sentences = list(set([string for string in df['X'].values]))
unique_words = list(set([word for string in df['X'].values for word in string.split(' ')]))
unique_words_sorted = sorted(unique_words)
#print(len(unique_words))
#print(unique_words[0:100])
#print(' ' in unique_sentences)
#print('' in unique_words)
#print(all([not string.isspace() for string in unique_words]))
#print(np.any(dftr["X"].values == ' '))
print(np.any([[word for word in sentence] for sentence in dftr["X"].values] == ' '))

[50, 47, 46, 46, 44, 44, 44, 44, 43, 43, 43, 42, 42, 41, 41, 41, 41, 41, 40, 40, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35]
False


In [8]:
#Q1
def bag_of_words(tweets_array):
    vectorizer = CountVectorizer(vocabulary=unique_words)
    X = vectorizer.fit_transform(tweets_array)
    
    return X.astype(np.bool_)

#bag_of_words(dftr["X"].values)

In [9]:
#Q2
def tf_idf(tweets_array):
    
    tfidf = TfidfVectorizer(vocabulary=np.array(unique_words))

    tfidf.fit(tweets_array)
    X = tfidf.transform(tweets_array)

    return X

In [10]:
#Q3
def word_2_vec_skipgram(tweets_array, agg):
    
    X = [np.float16(agg([wv_word2vec[word] if word in wv_word2vec else default_word2vec for word in sentence], axis=0)) for sentence in tweets_array]

    return X

#print(word_2_vec_skipgram(dftr["X"].values));

In [11]:
#Q4
def glove_emb(tweets_array, agg):
    #print(np.any([[word for word in sentence] for sentence in tweets_array] == ' '))
    X = [np.float16(agg([wv_glove[word] if word in wv_glove else default_glove for word in sentence], axis=0)) for sentence in tweets_array]
    return X

#print(word_2_vec_skipgram(dftr["X"].values));

In [12]:
#Q5
def fast_text(tweets_array, agg):
    X = [np.float16(agg([wv_fasttext[word] if word in wv_fasttext else default_word2vec for word in sentence], axis=0)) for sentence in tweets_array]
    return X
    

#fast_text(dftr["X"].values)


In [13]:
#Q6

In [14]:
#Q7
agg1 = np.mean
agg2 = np.maximum.reduce
agg3 = np.minimum.reduce
# max_words = 50
# def agg3(l_vec, axis):
#     ir = np.concatenate(l_vec)
#     #pad with extra zeros if smaller
#     print(max_words*l_vec[0].shape[0])
#     print(ir.shape[0])
#     return np.concatenate((np.array(ir), np.zeros(max_words*l_vec[0].shape[0] - ir.shape[0], dtype=type(ir[0]))))
    

word2vec_avg = lambda x : word_2_vec_skipgram(x, agg=agg1)
word2vec_avg.__name__ = "word2vec_avg"
word2vec_max = lambda x : word_2_vec_skipgram(x, agg=agg2)
word2vec_max.__name__ = "word2vec_max"
word2vec_min = lambda x : word_2_vec_skipgram(x, agg=agg3)
word2vec_min.__name__ = "word2vec_min"

glove_avg = lambda x : glove_emb(x, agg=agg1)
glove_avg.__name__ = "glove_avg"
glove_max = lambda x : glove_emb(x, agg=agg2)
glove_max.__name__ = "glove_max"
glove_min = lambda x : glove_emb(x, agg=agg3)
glove_min.__name__ = "glove_min"

fasttext_avg = lambda x : fast_text(x, agg=agg1)
fasttext_avg.__name__ = "fasttext_avg"
fasttext_max = lambda x : fast_text(x, agg=agg2)
fasttext_max.__name__ = "fasttext_max"
fasttext_min = lambda x : fast_text(x, agg=agg3)
fasttext_min.__name__ = "fasttext_min"

In [15]:
#Q8

#print(dftr)
# dfte = pd.read_parquet("data/test.parquet")
# dfvl = pd.read_parquet("data/valid.parquet")
#test_embedding_matrix = np.random.randint(0, high=1, size=(dftr.shape[0] + dfte.shape[0], len(unique_words)), dtype="int")
seed = 1234
embeddings = [bag_of_words, tf_idf, word2vec_avg, word2vec_max, word2vec_min, glove_avg, glove_max, glove_min, fasttext_avg, fasttext_max, fasttext_min]

dftr = pd.read_parquet("data/train.parquet")
dfte = pd.read_parquet("data/test.parquet")
dfvl = pd.read_parquet("data/valid.parquet")


#classifiers = [svm.SVC, LogisticRegression, RandomForestClassifier]
classifiers = ["SVM", "LogRegr", "RandomForest"]
ys = ["positive", "negative"]
X_train_raw = dftr["X"].str.replace("\u00A0","").values
X_test_raw = dfte["X"].str.replace("\u00A0","").values
X_valid_raw = dfvl["X"].str.replace("\u00A0","").values

rows = [e.__name__ + "_" + c + "_" + y for e, c, y in itertools.product(*[embeddings, classifiers, ys])]
cols = ["f1_1", "f1_2", "f1_3", "f1_4", "f1_5", "f1_avg", "duration"]
df_res = pd.DataFrame(index=rows, columns=cols)
#df_res.loc[str(word_2_vec_skipgram), str(svm.SVC) +dd "_positive_f1"] = 1.0
#print(df_res.loc[str(word_2_vec_skipgram), str(svm.SVC) + "_positive_f1"])

for e in embeddings[8:]:
    print(f"trying embedding {e.__name__}")
    X_train = e(X_train_raw)
    X_test = e(X_test_raw)
    X_valid = e(X_valid_raw)
    
    for c in classifiers:
        

        for y in ys:
            #print(f"trying embedding {str(e)} with classifier {c} for analysis of {y} Sentiment")
            start_time = time.perf_counter()
            match c:
                case "SVM":
                    clf = svm.SVC(random_state=seed, kernel="rbf", gamma='scale', class_weight='balanced', C=5.0)
                case "LogRegr":
                    clf = LogisticRegression(random_state=seed, C=5.0, penalty='l2', solver='saga', max_iter=1000, class_weight='balanced')
                case "RandomForest":
                    clf = RandomForestClassifier(random_state=seed, max_depth=20, min_samples_split=0.2, min_samples_leaf=0.1, max_features='log2', class_weight="balanced")
            

            y_train = dftr["y_" + y[0:3]].values
            #print(dftr["y_" + y[0:3]].unique())
            y_test = dfte["y_" + y[0:3]].values
            #print(y_test)
            #print(f"starting to fit for {c}")
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            
           
            #print('Precision: %.3f' % precision_score(y_test, y_pred, average=None))
            #print('Recall: %.3f' % recall_score(y_test, y_pred, average=None))
            f1 = f1_score(y_test, y_pred, average=None)
            f1_avg = f1_score(y_test, y_pred, average="macro")            
            
            row_name = e.__name__ + "_" + c + "_" + y
            #print(row_name)
            #print(f1.shape)
            #print("f1 score: ", f1)
            df_res.loc[row_name, cols[0:f1.shape[0]]] = f1
            df_res.loc[row_name, "f1_avg"] = f1_avg
            
            end_time = time.perf_counter()
            df_res.loc[row_name, "duration"] = np.round(end_time-start_time, 2)
            df_res.to_csv("data/q8res.csv")
            #print('Accuracy: %.3f' % accuracy_score(y_test, y_pred, average=None))


        

trying embedding fasttext_avg


In [ ]:
X = [[0], [1], [2], [3]]
Y = np.array([0, 1, 2, 3]).astype(np.int8)
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X, Y)
svm.SVC(decision_function_shape='ovo')
dec = clf.decision_function([[1]])
dec.shape[1] # 4 classes: 4*3/2 = 6
clf.decision_function_shape = "ovr"
dec = clf.decision_function([[1]])
dec.shape[1] # 4 classes


4